In [36]:
# to run async code in jupyter notebook
import nest_asyncio
nest_asyncio.apply()

In [37]:
# Please change the following to your own PAPER api key and secret
# or set them as environment variables (ALPACA_API_KEY, ALPACA_SECRET_KEY).
# You can get them from https://alpaca.markets/

API_KEY = None
SECRET_KEY = None

#### We use paper environment for this example ####
paper = True # Please do not modify this. This example is for paper trading only.
####

# Below are the variables for development this documents
# Please do not change these variables
trade_api_url = None
trade_api_wss = None
data_api_url = None
stream_data_wss = None

In [38]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

if API_KEY is None:
    API_KEY = os.environ.get('ALP_API_KEY')

if SECRET_KEY is None:
    SECRET_KEY = os.environ.get('ALP_SEC_KEY')

In [39]:
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

from alpaca.trading.client import TradingClient
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.historical.corporate_actions import CorporateActionsClient
from alpaca.data.historical.stock import StockHistoricalDataClient
from alpaca.trading.stream import TradingStream
from alpaca.data.live.stock import StockDataStream

from alpaca.data.requests import (
    CorporateActionsRequest,
    StockBarsRequest,
    StockQuotesRequest,
    StockTradesRequest,
)
from alpaca.trading.requests import (
    ClosePositionRequest,
    GetAssetsRequest,
    GetOrdersRequest,
    LimitOrderRequest,
    MarketOrderRequest,
    StopLimitOrderRequest,
    StopLossRequest,
    StopOrderRequest,
    TakeProfitRequest,
    TrailingStopOrderRequest,
)
from alpaca.trading.enums import (
    AssetExchange,
    AssetStatus,
    OrderClass,
    OrderSide,
    OrderType,
    QueryOrderStatus,
    TimeInForce,
)

In [40]:
# setup clients
trade_client = TradingClient(api_key=API_KEY, secret_key=SECRET_KEY, paper=paper, url_override=trade_api_url)

In [41]:
# check trading account
# You can check definition of each field in the following documents
# ref. https://docs.alpaca.markets/docs/account-plans
# ref. https://docs.alpaca.markets/reference/getaccount-1
acct = trade_client.get_account()
acct

{   'account_blocked': False,
    'account_number': 'PA324WWLFGIT',
    'accrued_fees': '0',
    'buying_power': '200000',
    'cash': '100000',
    'created_at': datetime.datetime(2025, 6, 21, 9, 19, 27, 520093, tzinfo=TzInfo(UTC)),
    'crypto_status': <AccountStatus.ACTIVE: 'ACTIVE'>,
    'currency': 'USD',
    'daytrade_count': 0,
    'daytrading_buying_power': '0',
    'equity': '100000',
    'id': UUID('5b316923-d090-40c1-b5b2-35a61aebc6a3'),
    'initial_margin': '0',
    'last_equity': '100000',
    'last_maintenance_margin': '0',
    'long_market_value': '0',
    'maintenance_margin': '0',
    'multiplier': '2',
    'non_marginable_buying_power': '100000',
    'options_approved_level': 3,
    'options_buying_power': '100000',
    'options_trading_level': 3,
    'pattern_day_trader': False,
    'pending_transfer_in': None,
    'pending_transfer_out': None,
    'portfolio_value': '100000',
    'regt_buying_power': '200000',
    'short_market_value': '0',
    'shorting_enabled': 

In [42]:
# check account configuration
# ref. https://docs.alpaca.markets/reference/getaccountconfig-1
acct_config = trade_client.get_account_configurations()
acct_config

{   'dtbp_check': <DTBPCheck.ENTRY: 'entry'>,
    'fractional_trading': True,
    'max_margin_multiplier': '4',
    'max_options_trading_level': None,
    'no_shorting': False,
    'pdt_check': <PDTCheck.ENTRY: 'entry'>,
    'ptp_no_exception_entry': False,
    'suspend_trade': False,
    'trade_confirm_email': <TradeConfirmationEmail.ALL: 'all'>}

In [43]:
# set account configuration
# ref. https://docs.alpaca.markets/reference/patchaccountconfig-1
req = acct_config
req.fractional_trading = not req.fractional_trading # toggle fractional trading
acct_config_new = trade_client.set_account_configurations(req)
display(acct_config_new)

# revert changes
req = acct_config_new
req.fractional_trading = not req.fractional_trading # toggle fractional trading
acct_config_reverted = trade_client.set_account_configurations(req)
display(acct_config_reverted)

{   'dtbp_check': <DTBPCheck.ENTRY: 'entry'>,
    'fractional_trading': False,
    'max_margin_multiplier': '4',
    'max_options_trading_level': None,
    'no_shorting': False,
    'pdt_check': <PDTCheck.ENTRY: 'entry'>,
    'ptp_no_exception_entry': False,
    'suspend_trade': False,
    'trade_confirm_email': <TradeConfirmationEmail.ALL: 'all'>}

{   'dtbp_check': <DTBPCheck.ENTRY: 'entry'>,
    'fractional_trading': True,
    'max_margin_multiplier': '4',
    'max_options_trading_level': None,
    'no_shorting': False,
    'pdt_check': <PDTCheck.ENTRY: 'entry'>,
    'ptp_no_exception_entry': False,
    'suspend_trade': False,
    'trade_confirm_email': <TradeConfirmationEmail.ALL: 'all'>}

In [44]:
# setup stock historical data client
stock_historical_data_client = StockHistoricalDataClient(API_KEY, SECRET_KEY, url_override = data_api_url)

In [45]:
# we will place orders for symbol: SPY in this example
symbol = "SPY"

In [46]:
# get historical bars by symbol
# ref. https://docs.alpaca.markets/reference/stockbars-1
now = datetime.now(ZoneInfo("America/New_York"))
req = StockBarsRequest(
    symbol_or_symbols = [symbol],
    timeframe=TimeFrame(amount = 1, unit = TimeFrameUnit.Hour), # specify timeframe
    start = now - timedelta(days = 5),                          # specify start datetime, default=the beginning of the current day.
    # end_date=None,                                        # specify end datetime, default=now
    limit = 2,                                               # specify limit
)
stock_historical_data_client.get_stock_bars(req).df

open    high     low   close    volume  \
symbol timestamp                                                               
SPY    2025-06-16 23:00:00+00:00  600.8383  601.50  598.92  598.94  523997.0   
       2025-06-17 08:00:00+00:00  599.0700  600.24  598.84  599.83  197126.0   

                                  trade_count        vwap  
symbol timestamp                                           
SPY    2025-06-16 23:00:00+00:00       3879.0  600.047878  
       2025-06-17 08:00:00+00:00       2609.0  599.567118

In [47]:
# get historical trades by symbol
req = StockTradesRequest(
    symbol_or_symbols = [symbol],
    start = now - timedelta(days = 5),                          # specify start datetime, default=the beginning of the current day.
    # end=None,                                             # specify end datetime, default=now
    limit = 2,                                                # specify limit
)
stock_historical_data_client.get_stock_trades(req).df

exchange   price  size  \
symbol timestamp                                                 
SPY    2025-06-16 22:17:57.023266+00:00        P  602.44  50.0   
       2025-06-16 22:17:57.023455+00:00        Z  602.44  13.0   

                                                     id    conditions tape  
symbol timestamp                                                            
SPY    2025-06-16 22:17:57.023266+00:00  52983611015491  [ , F, T, I]    B  
       2025-06-16 22:17:57.023455+00:00  52983527338290  [ , F, T, I]    B

In [48]:
# get historical quotes by symbol
req = StockQuotesRequest(
    symbol_or_symbols = [symbol],
    start = now - timedelta(days = 5),                      # specify start datetime, default=the beginning of the current day.
    # end=None,                                             # specify end datetime, default=now
    limit = 2,                                              # specify limit
)
stock_historical_data_client.get_stock_quotes(req).df

bid_price  bid_size bid_exchange  \
symbol timestamp                                                            
SPY    2025-06-16 22:17:34.318029+00:00     602.42       1.0            P   
       2025-06-16 22:17:34.323923+00:00     602.42       1.0            P   

                                         ask_price  ask_size ask_exchange  \
symbol timestamp                                                            
SPY    2025-06-16 22:17:34.318029+00:00     602.46       5.0            P   
       2025-06-16 22:17:34.323923+00:00     602.47       5.0            P   

                                        conditions tape  
symbol timestamp                                         
SPY    2025-06-16 22:17:34.318029+00:00        [R]    B  
       2025-06-16 22:17:34.323923+00:00        [R]    B

In [50]:
stock_data_stream_client = StockDataStream(API_KEY, SECRET_KEY, url_override = stream_data_wss)

async def stock_data_stream_handler(data):
    print(data)

symbols = [symbol]

stock_data_stream_client.subscribe_quotes(stock_data_stream_handler, *symbols)
stock_data_stream_client.subscribe_trades(stock_data_stream_handler, *symbols)

stock_data_stream_client.run()

error during websocket communication: connection limit exceeded
Traceback (most recent call last):
  File "c:\Users\sergi\Documents\TFG\.venv\Lib\site-packages\alpaca\data\live\websocket.py", line 343, in _run_forever
    await self._start_ws()
  File "c:\Users\sergi\Documents\TFG\.venv\Lib\site-packages\alpaca\data\live\websocket.py", line 135, in _start_ws
    await self._auth()
  File "c:\Users\sergi\Documents\TFG\.venv\Lib\site-packages\alpaca\data\live\websocket.py", line 126, in _auth
    raise ValueError(msg[0].get("msg", "auth failed"))
ValueError: connection limit exceeded
error during websocket communication: connection limit exceeded
Traceback (most recent call last):
  File "c:\Users\sergi\Documents\TFG\.venv\Lib\site-packages\alpaca\data\live\websocket.py", line 343, in _run_forever
    await self._start_ws()
  File "c:\Users\sergi\Documents\TFG\.venv\Lib\site-packages\alpaca\data\live\websocket.py", line 135, in _start_ws
    await self._auth()
  File "c:\Users\sergi\Docu

keyboard interrupt, bye


TimeoutError: 

In [ ]:
corporate_actions_client = CorporateActionsClient(API_KEY, SECRET_KEY)
corporate_actions_client.get_corporate_actions(CorporateActionsRequest(
    start=datetime(2020, 1, 1),
    symbols=[symbol]
)).df

,symbol,rate,special,foreign,process_date,ex_date,record_date,payable_date
corporate_action_type,,,,,,,,
cash_dividends,SPY,1.405559,False,False,2020-04-30,2020-03-20,2020-03-23,2020-04-30
cash_dividends,SPY,1.366242,False,False,2020-07-31,2020-06-19,2020-06-22,2020-07-31
cash_dividends,SPY,1.339224,False,False,2020-10-30,2020-09-18,2020-09-21,2020-10-30
cash_dividends,SPY,1.580000,False,False,2021-01-29,2020-12-18,2020-12-21,2021-01-29
cash_dividends,SPY,1.277788,False,False,2021-04-30,2021-03-19,2021-03-22,2021-04-30
cash_dividends,SPY,1.375875,False,False,2021-07-30,2021-06-18,2021-06-21,2021-07-30
cash_dividends,SPY,1.428117,False,False,2021-10-29,2021-09-17,2021-09-20,2021-10-29
cash_dividends,SPY,1.636431,False,False,2022-01-31,2021-12-17,2021-12-20,2022-01-31
cash_dividends,SPY,1.366009,False,False,2022-04-29,2022-03-18,2022-03-21,2022-04-29


In [52]:
from datetime import datetime, timedelta
client = StockHistoricalDataClient(API_KEY, SECRET_KEY)

request = StockBarsRequest(
    symbol_or_symbols=["AAPL"],
    timeframe=TimeFrame.Minute,
    start=datetime.utcnow() - timedelta(days=1),
    end=datetime.utcnow() - timedelta(minutes=20)  # exclude the most recent 20 min
)
bars = client.get_stock_bars(request)


C:\Users\sergi\AppData\Local\Temp\ipykernel_11588\2833964204.py:7: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  start=datetime.utcnow() - timedelta(days=1),
C:\Users\sergi\AppData\Local\Temp\ipykernel_11588\2833964204.py:8: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end=datetime.utcnow() - timedelta(minutes=20)  # exclude the most recent 20 min
